### 1. Set up a regression experiment

In [1]:
# !pip install interpret
import pandas as pd
from sklearn.model_selection import train_test_split
price = pd.read_csv('./processed-data/new_train.csv')
y = price['SalePrice']
X = price.drop(['SalePrice'],axis=1)
seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

### 2. Train the Explainable Boosting Machine (EBM)

#### default EBM

In [2]:
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree

ebm1 = ExplainableBoostingRegressor(random_state=seed)
ebm1.fit(X_train, y_train)

ExplainableBoostingRegressor(binning_strategy='quantile', data_n_episodes=2000,
                             early_stopping_run_length=50,
                             early_stopping_tolerance=1e-05,
                             feature_names=['MSZoning', 'LotFrontage',
                                            'LotArea', 'Street', 'Alley',
                                            'LotShape', 'LandContour',
                                            'Utilities', 'LotConfig',
                                            'LandSlope', 'Neighborhood',
                                            'Condition1', 'Condition2',
                                            'BldgType', 'HouseStyle',
                                            'OverallQual', 'Over...
                                            'categorical', 'categorical',
                                            'continuous', 'continuous',
                                            'continuous', 'categorical',
            

It can be seen that the default boosting parameters such as `learning_rate` is quite small and `n_estimators` captures the number of trees that we add to the model, we then explore its model performance in terms of ***RMSE*** using validation test set.

In [3]:
from interpret import show
from interpret.perf import RegressionPerf
ebm1_perf = RegressionPerf(ebm1.predict).explain_perf(X_test, y_test, name='EBM_default')
show(ebm1_perf)

<!-- http://127.0.0.1:7310/140616010802064/ -->

#### EBM with tuning n_estimators

We did some initial explorations changing the `n_estimators` to give us an intuitive about the approximate range of parameter setting,as it captured the number of trees that we add to the model. A high number of trees can be computationally expensive, so for the sake of computational efficiency, we need to narrow our range setting.

In [4]:
ebm2 = ExplainableBoostingRegressor(random_state=seed, n_estimators=50)
ebm2.fit(X_train, y_train)

ExplainableBoostingRegressor(binning_strategy='quantile', data_n_episodes=2000,
                             early_stopping_run_length=50,
                             early_stopping_tolerance=1e-05,
                             feature_names=['MSZoning', 'LotFrontage',
                                            'LotArea', 'Street', 'Alley',
                                            'LotShape', 'LandContour',
                                            'Utilities', 'LotConfig',
                                            'LandSlope', 'Neighborhood',
                                            'Condition1', 'Condition2',
                                            'BldgType', 'HouseStyle',
                                            'OverallQual', 'Over...
                                            'categorical', 'categorical',
                                            'continuous', 'continuous',
                                            'continuous', 'categorical',
            

In [5]:
ebm2_perf = RegressionPerf(ebm2.predict).explain_perf(X_test, y_test, name='EBM2')
show(ebm2_perf)

<!-- http://127.0.0.1:7310/140616084227072/ -->

In [6]:
ebm3 = ExplainableBoostingRegressor(random_state=seed, n_estimators=100)
ebm3.fit(X_train, y_train)

ExplainableBoostingRegressor(binning_strategy='quantile', data_n_episodes=2000,
                             early_stopping_run_length=50,
                             early_stopping_tolerance=1e-05,
                             feature_names=['MSZoning', 'LotFrontage',
                                            'LotArea', 'Street', 'Alley',
                                            'LotShape', 'LandContour',
                                            'Utilities', 'LotConfig',
                                            'LandSlope', 'Neighborhood',
                                            'Condition1', 'Condition2',
                                            'BldgType', 'HouseStyle',
                                            'OverallQual', 'Over...
                                            'categorical', 'categorical',
                                            'continuous', 'continuous',
                                            'continuous', 'categorical',
            

In [7]:
ebm3_perf = RegressionPerf(ebm3.predict).explain_perf(X_test, y_test, name='EBM3')
show(ebm3_perf)

<!-- http://127.0.0.1:7310/140616080282848/ -->

It can be seen that RMSE is decreasing at the range from 20 to 50 and increase at 100,so we set up the parameter range from 10 to 100, especially, we use `negative_mean_squared_error` as our scoring benchmark.`n_estimators `captures the number of trees that we add to the model. A high number of trees can be computationally expensive. 

In [8]:
from sklearn.model_selection import learning_curve, GridSearchCV

In [9]:
param_test1 = {'n_estimators':range(10,100,10)}
gsearch = GridSearchCV(estimator = ExplainableBoostingRegressor(learning_rate=0.01,random_state=seed), 
param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ExplainableBoostingRegressor(binning_strategy='quantile',
                                                    data_n_episodes=2000,
                                                    early_stopping_run_length=50,
                                                    early_stopping_tolerance=1e-05,
                                                    feature_names=None,
                                                    feature_step_n_inner_bags=0,
                                                    feature_types=None,
                                                    holdout_size=0.15,
                                                    holdout_split=0.15,
                                                    interactions=0,
                                                    learning_rate=0.01,
                                                    main_attr='all',
                                                  

In [13]:
gsearch.best_params_, gsearch.best_score_

({'n_estimators': 20}, -838617776.0883902)

It can be seen that we get the best score when ***`n_parameters` = 20***. So we train our best model so-far and display its performance at the next step.

In [14]:
ebm_best = ExplainableBoostingRegressor(random_state=seed, n_estimators=20)
ebm_best.fit(X_train, y_train)

ExplainableBoostingRegressor(binning_strategy='quantile', data_n_episodes=2000,
                             early_stopping_run_length=50,
                             early_stopping_tolerance=1e-05,
                             feature_names=['MSZoning', 'LotFrontage',
                                            'LotArea', 'Street', 'Alley',
                                            'LotShape', 'LandContour',
                                            'Utilities', 'LotConfig',
                                            'LandSlope', 'Neighborhood',
                                            'Condition1', 'Condition2',
                                            'BldgType', 'HouseStyle',
                                            'OverallQual', 'Over...
                                            'categorical', 'categorical',
                                            'continuous', 'continuous',
                                            'continuous', 'categorical',
            

In [15]:
ebm_best_perf = RegressionPerf(ebm_best.predict).explain_perf(X_test, y_test, name='EBM_best')
show(ebm_best_perf)

<!-- http://127.0.0.1:7310/140616061711584/ -->

#### Save the prediction results

In [16]:
validation = pd.read_csv('./processed-data/new_test.csv')
valid_x = validation.drop(['SalePrice'],axis=1)
preds = ebm_best.predict(valid_x)
final_preds = pd.DataFrame(preds)
raw_valid = pd.read_csv('house-prices-data/test.csv')
raw_id = raw_valid['Id']
output = pd.concat([raw_id, final_preds], axis=1)
output = output.rename(columns={'predict': "SalePrice"})

In [17]:
output.to_csv('results/EBM_results.csv', index=False)

### 3. Global Explanations: What the model learned overall

In [18]:
ebm_global = ebm_best.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7310/140616079783248/ -->

1. Choose ***Summary*** to show the overall variable importance ranked in descending order (orange color).

2. Choose the first variable `MSZoning`. Two plots show up: the Partial Dependent Plot (PDP) and the histogram of “MSZoning”. The histogram indicates most of the zoning classification is `RL`(Residential Low Density). The PDP presents the marginal effect of the feature on the predicted outcome of a machine learning model. It tells whether the relationship between the target and a feature is linear, monotonic or more complex. In this example the PDP shows when the zoning classification is `C`(commercial), there is a ***negative*** effect on the `SalePrice`.

### 4. Local Explanations: How an individual prediction was made

In [19]:
ebm_local = ebm_best.explain_local(X_test[:10], y_test[:10], name='EBM')
show(ebm_local)

<!-- http://127.0.0.1:7310/140616004346880/ -->

The drop-down menu lists the predicted value and the actual value for each record. We choose the first record. The value of `GrLivArea` is 420, and that of `LowQualFinSF` is 4, and so on. The contributions of all variables for this record are ranked in descending order as below. `GrLivArea` positively contributes to the target `SalePrice`, while `LowQualFinSF` negatively contributes to the target. Because EBM is a GAM-like model, the prediction is the sum of all the coefficients.

### 5. Test out a few other Explainable Models

In [21]:
X = pd.get_dummies(data=X, drop_first=True)
seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

lr = LinearRegression(random_state=seed)
lr.fit(X_train, y_train)

rt = RegressionTree(random_state=seed)
rt.fit(X_train, y_train)

/Users/scarlettwang/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 253579961217.50323, tolerance: 711570848.3106698



In [22]:
lr_perf = RegressionPerf(lr.predict).explain_perf(X_test, y_test, name='Linear Regression')
rt_perf = RegressionPerf(rt.predict).explain_perf(X_test, y_test, name='Regression Tree')

show(lr_perf)
show(rt_perf)
show(ebm_best_perf)

<!-- http://127.0.0.1:7310/140616099676624/ -->

<!-- http://127.0.0.1:7310/140616099676904/ -->

<!-- http://127.0.0.1:7310/140616061711584/ -->

The R-squared value of EBM is 0.89 which outperforms those of linear regression model and regression tree model.In terms of both R-squared and RMSE, we could conclude that EBM preform the best.

### 6. Glassbox: All of our models have global and local explanations

In [23]:
lr_global = lr.explain_global(name='Linear Regression')
rt_global = rt.explain_global(name='Regression Tree')

show(lr_global)
show(rt_global)
show(ebm_global)

<!-- http://127.0.0.1:7310/140616093022416/ -->

<!-- http://127.0.0.1:7310/140616093023200/ -->

<!-- http://127.0.0.1:7310/140616079783248/ -->

### 7. Dashborad for all 3 models

In [24]:
show([lr_global, lr_perf, rt_global, rt_perf, ebm_global, ebm_best_perf])

<!-- http://127.0.0.1:7310/140616087297992/ -->
 Open in new window